In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports
import pandas as pd
import dash_leaflet as dl
from dash import Dash, dcc, html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output

import numpy as np
import base64

!pip install jupyter-dash dash --quiet
# Import your CRUD module (adjust filename/classname to match YOUR Project One)
from CRUD_Python_Module import AnimalShelter  # example

# Create the DB connection with hardcoded aacuser credebtials

USERNAME ="aacuser"
PASSWORD = "ab@0ne"

# Your DB/collection names may be "aac" and "animals" depending on the SNHU Codio setup
DB_NAME = "aac"
COLLECTION = "animals"  

shelter = AnimalShelter(USERNAME, PASSWORD, DB_NAME, COLLECTION)

# Retrieve all documents from ongoDB
data = shelter.read({})  # <-- change method name if needed
                        
# Convert to DataFrame
df = pd.DataFrame(shelter.read({}))
                        
# Optional but highly recommended: remove MongoDB_id so the table renders cleanly
if "_id" in df.columns:
    df.drop(columns=["_id"], inplace=True)
                    
df.head()
                    
app = JupyterDash(DB_NAME)

# Your unique identifier from Module Five 
UNIQUE_ID = "BeverlyVaughan-Module5-UNIQUEID"
app.layout = html.Div([
    html.H1("Grazioso Salvare Dashboard"),
   # html.P("If you can see this, the server is working.")
    html.H4(UNIQUE_ID),
        
    html.Hr(),
            
    # Data table
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        
        # Required for map callback
        row_selectable="single",   # or "multi"
        selected_rows=[0],         # default selection (first row)
                 
        # Helpful usability options
        page_size=10,
        sort_action="native",
        filter_action="native",
        style_table={'overflowX': 'auto'},
        style_cell={'textAlign': 'left', 'padding': '5px', 'whiteSpace': 'normal'},
    ),
    html.Br(),
        
    # Map placeholder (the callback will populate this)
    html.Div(
        id='map-id',
        className='col s12 m6',
        
    )
])
            
###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name. NOTE: You will
# likely need more variables for your constructor to handle the hostname and port of the MongoDB
# server, and the database and collection names

# Hardcode credentials per milestone instructions



# Your DB/collection names may be "aac" and "animals" depending on the SNHU Codio setup
DB_NAME = "aac"
COLLECTION = "animals"

shelter = AnimalShelter(USERNAME, PASSWORD, DB_NAME, COLLECTION)

# Retrieve all documents from MongoDB
# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
data = shelter.read({}) # <-- change method if needed

# Convert to DataFrame
df = pd.DataFrame.from_records(shelter.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)

if "_id" in df.columns:
    df.drop(columns=["_id"], inplace=True)
    
df.head()

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(DB_NAME)

UNIQUE_ID = "BeverlyVaughan-Module5-UNIQUEID"  

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Hr(),
    
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        #FIXME: Set up the features for your interactive data table to make it user-friendly for your client
        editable = False,
        filter_action = "native",
        sort_action = "native",
        sort_mode = "multi",
        column_selectable = "single",
        row_selectable="single",   # or "multi"
        selected_rows=[0],         # default selection (first row)
        row_deletable = False,
        selected_columns = [],
        page_action = "native",
        page_current = 0,
        # Helpful usability options
        page_size = 10,
        style_table={'overflowX' : 'auto'},
        style_cell={'textAliign': 'left', 'padding': '5px', 'whiteSpace': 'normal'},
                    
    ),
    html.Br(),
    
    html.Center(html.B(html.H2("Thank You"))),
    html.Center(html.B(html.H2("Enjoy"))),
    
    
    html.Hr(),
    html.Div(
            id='map-id',
            className='col s12 m6',
            )
])

#############################################
# Interaction Between Components / Controller
#############################################
#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('map-id', 'children'),
    [Input('datatable-id', 'derived_viewport_data'),
     Input('datatable_id', 'selected_rows')]
) 
def update_map_callback(viewData, selected_rows):
    # derived_viewport_data = what the table is currently showing (after filter/sort)
    # selected_rows = List Like [0]
    if viewData is none or len(viewData) == 0:
         return []
    return update_map(viewData, selected_rows)
            
def on_click(button, button2):

    # start case
    df = pd.DataFrame.from_records(db.read({}))
    
    # use highernumber of buttons clicks to determine filter type.
    if (int(button) > int(button2)):
        
        df = pd.DataFrame.from_records(db.read({"animal_type" : "Cat"}))
    elif (int(button2) > int(button1)):
        df = pd.DataFrame.from_records(db.read({"animal_type" : "Dog"}))
        
    # cleanup Mongo _id field
    df.drop(columns=["_id"], inplace=True)
    return df.to_dict('records')
        
        
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data"),
     Input('datatable-id', "selected_rows")]
)  
            
def update_map(viewData, index):
    dff = pd.DataFrame.from_dict(viewData)
                    
    # Because we only allow single row selection, the list can be converted to a raw index here
    if index is None:
        row = 0
    else:
        row = index[0]   
    
#FIXME Add in the code for your geolocation chart
    # Austin TX is at [30.75-97.48]                
    return [
        dl.Map(
            style={'width' : '1000px', 'height': '500px'},
            center=[30.75,-97.48], 
            zoom=10, 
            children=[
                dl.TileLayer(id="base-layer-id"),
                   
                   # Marker with tool tip and popup
                   # Column 13 and 14 define the grid coordinates for the map
                   # Column 4 defines the breed for the animal
                   # Column 9 defines the name of the animal
                dl.Marker(
                    position=[dff.iloc[row,13], dff.iloc[row,14]],
                    children=[
                        dl.ToolTip(dff.iloc[row]['breed'] if 'breed' in ddff.columns else 'Unknown'),
                        dl.Popup([
                                     
                             html.H1("Animal Name"),
                             html.P(dff.iloc[row]['name'] if 'name' in dff.columns else 'Unknown')
                                         
                        ])
                               
                    ]
                           
                )           
                
            ]        
                
        )
               
]
        
    
                    
# Run app and display result in jupyterlab mode, note, if you have previously run a prior app, the default port of 8050 may not be available, if so, try setting an alternate port.
#if ___name___ == "_main_":
app.run_server(mode="jupyterlab", debug=False, port=8060)
    


[notice] A new release of pip is available: 23.1.2 -> 26.0.1
[notice] To update, run: pip install --upgrade pip


 * Running on http://127.0.0.1:8060/ (Press CTRL+C to quit)
127.0.0.1 - - [10/Feb/2026 04:13:15] "GET /_alive_2688d703-48ce-4656-8232-6dadd704110b HTTP/1.1" 200 -
